In [ ]:
import pandas as pd
HC3_filtered = pd.concat([pd.read_hdf('/content/drive/MyDrive/HC3 - filtered/GPT_df_3.h5'), pd.read_hdf('/content/drive/MyDrive/HC3 - filtered/Human_df_3.h5')], axis=0).fillna(0)
GPT_2 = pd.concat([pd.read_hdf('/content/drive/MyDrive/GPT2/GPT_df_gpt2.h5'), pd.read_hdf('/content/drive/MyDrive/GPT2/Human_df_gpt2.h5')], axis=0).fillna(0)
HC3_unfiltered = pd.concat([pd.read_hdf('/content/drive/MyDrive/HC3 - Unfiltered/GPT_df_3.h5'), pd.read_hdf('/content/drive/MyDrive/HC3 - Unfiltered/Human_df_3.h5')], axis=0).fillna(0)


In [ ]:
# Replacement dictionary
replacement_dict = {
    'Human_train': 'train',
    'Human_test': 'test',
    'Human_val': 'val',
    'GPT_train': 'train',
    'GPT_test': 'test',
    'GPT_val': 'val'
}

# Apply the replacements
GPT_2 = GPT_2.replace(replacement_dict)

In [ ]:
# Create the datasets dictionary
datasets = {
    'HC3_filtered': HC3_filtered,
    'HC3_unfiltered': HC3_unfiltered,
     'GPT_2': GPT_2
}

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, f1_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


f1_results = []

for train_name, train_df in datasets.items():
    # Define the columns based on the training dataset
    feature_cols = train_df.drop(['source', 'text', 'label'], axis=1).columns.tolist()

    # Prepare train data
    X_train = train_df[train_df['source'].isin(['train'])][feature_cols]
    y_train = train_df[train_df['source'].isin(['train'])]['label']

    X_val = train_df[train_df['source'].isin(['val'])][feature_cols]
    y_val = train_df[train_df['source'].isin(['val'])]['label']

    # Train model
    clf = xgb.XGBClassifier(eval_metric="error", early_stopping_rounds=15)
    evals = [(X_train, y_train), (X_val, y_val)]
    clf.fit(X_train, y_train, eval_set=evals, verbose=False)

    for test_name, test_df in datasets.items():
        # Ensure test dataset has same columns as train dataset
        X_test = test_df[test_df['source'].isin(['test'])][feature_cols]
        y_test = test_df[test_df['source'].isin(['test'])]['label']

        # Predict
        y_pred = clf.predict(X_test)

        # Compute F1 score (binary classification)
        f1 = f1_score(y_test, y_pred)

        # Append results to the f1_results list
        f1_results.append((train_name, test_name, f1))

        print(f"Training on {train_name}, Testing on {test_name}")
        print(classification_report(y_test, y_pred))

Training on HC3_filtered, Testing on HC3_filtered
              precision    recall  f1-score   support

           0       0.97      0.98      0.97     16148
           1       0.96      0.93      0.94      8012

    accuracy                           0.96     24160
   macro avg       0.96      0.96      0.96     24160
weighted avg       0.96      0.96      0.96     24160

Training on HC3_filtered, Testing on HC3_unfiltered
              precision    recall  f1-score   support

           0       0.96      0.97      0.97     16580
           1       0.94      0.92      0.93      8014

    accuracy                           0.96     24594
   macro avg       0.95      0.95      0.95     24594
weighted avg       0.96      0.96      0.96     24594

Training on HC3_filtered, Testing on GPT_2
              precision    recall  f1-score   support

           0       0.51      0.93      0.66      5000
           1       0.64      0.12      0.20      5000

    accuracy                         

In [ ]:
# Convert f1_results to DataFrame
df_f1 = pd.DataFrame(f1_results, columns=["Training Set", "Testing Set", "F1 Score"])
print(df_f1)

     Training Set     Testing Set  F1 Score
0    HC3_filtered    HC3_filtered  0.944307
1    HC3_filtered  HC3_unfiltered  0.934174
2    HC3_filtered           GPT_2  0.200978
3  HC3_unfiltered    HC3_filtered  0.942168
4  HC3_unfiltered  HC3_unfiltered  0.941886
5  HC3_unfiltered           GPT_2  0.220336
6           GPT_2    HC3_filtered  0.570016
7           GPT_2  HC3_unfiltered  0.561056
8           GPT_2           GPT_2  0.817926
